# <font color = 'indian red'> ***Final Pipeline***

# <font color = 'indian red'> **Install/Import Modules**

In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install transformers -U -qq
    !pip install sentencepiece -U -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.5 MB/s eta 0:00:00


In [ ]:
# Import the pandas library for data manipulation and analysis
import pandas as pd

# Import the numpy library for numerical computing
import numpy as np

# Import the Matplotlib library for creating visualizations such as plots, graphs, etc.
import matplotlib.pyplot as plt

# Import the pathlib library for working with file paths in a way that is cross-platform
from pathlib import Path

# Import functions for metrics computation like confusion matrix, and accuracy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, classification_report

# Import the transformers library for state-of-the-art Natural Language Processing (NLP) models like BERT, GPT, etc.
from transformers import pipeline

# Import the BeautifulSoup module for parsing HTML and XML documents
from bs4 import BeautifulSoup

# Import the re module for regular expressions
import re

In [ ]:
!pip show transformers

Name: transformers
Version: 4.33.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
# importing the libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# <font color = 'indian red'>**Specify Base folder for Project**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  base_folder = Path('/content/drive/MyDrive/Colab Notebooks') # MAKE SURE TO CHANGE THE PATH

# (the else block is required only if you have local GPU machine, other wise you can ignore the else block)
else:
  base_folder = Path('/content/drive/MyDrive/Colab Notebooks') # MAKE SURE TO CHANGE THE PATH

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[link text](https://)
# <font color = 'indian red'>**Create a Data Frame**

In [ ]:
data_folder = base_folder/'Colab Notebooks/'

In [ ]:
data_file = data_folder /'spam.csv'

In [ ]:
# creating Pandas Dataframe
spam_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spam.csv', encoding='ISO-8859-1')

In [ ]:
spam_df.info

<bound method DataFrame.info of         v1                                                 v2 Unnamed: 2  \
0      ham  Go until jurong point, crazy.. Available only ...        NaN   
1      ham                      Ok lar... Joking wif u oni...        NaN   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3      ham  U dun say so early hor... U c already then say...        NaN   
4      ham  Nah I don't think he goes to usf, he lives aro...        NaN   
...    ...                                                ...        ...   
5567  spam  This is the 2nd time we have tried 2 contact u...        NaN   
5568   ham              Will Ì_ b going to esplanade fr home?        NaN   
5569   ham  Pity, * was in mood for that. So...any other s...        NaN   
5570   ham  The guy did some bitching but I acted like i'd...        NaN   
5571   ham                         Rofl. Its true to its name        NaN   

     Unnamed: 3 Unnamed: 4  
0           NaN        NaN

In [ ]:
spam_df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
#Loading columns 1 & 2 into the spam dataframe
spam_df = spam_df.iloc[:,0:2]

In [ ]:
spam_df.rename(columns={'v1':'label','v2': 'message'}, inplace=True)

# <font color = 'indian red'> **Pre-Processing**

In [ ]:
#Pre-Processing
#Step 1: Remove HTML tags and new line characters

#Function to remove HTML tags
def html_tags(txt):
    soup = BeautifulSoup(txt, "html.parser")
    txt = soup.get_text()
    return txt

In [ ]:
spam_df['clean_text'] = spam_df['message'].apply(html_tags)

In [ ]:
#Step 2: Removing punctuations using spacy
!pip install -U spacy -qq

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
spam_df['clean_text'] = spam_df['clean_text'].apply(lambda text: ' '.join([token.text for token in nlp(text) if not token.is_punct]))

# <font color = 'indian red'> **Feature Engineering:**


In [ ]:
#1 Number of words
spam_df['Number_of_words'] = spam_df['clean_text'].apply(lambda x: len(x.split()))

In [ ]:
#2 Number of characters
spam_df['Number_of_characters'] = spam_df['clean_text'].apply(lambda x: len(x))

In [ ]:
#3 Number of digits
# Define a function to count the number of numbers
def count_numbers(text):
    words = text.split()
    numeric_count = sum(1 for word in words if word.isnumeric())
    return numeric_count

In [ ]:
spam_df['Number_of_digits'] = spam_df['clean_text'].apply(lambda x: count_numbers(x))

In [ ]:
# Define a function
def count_exclamation_marks(text):
    return text.count('!')

spam_df['Number_of_exclamation_marks'] = spam_df['clean_text'].apply(lambda x: count_exclamation_marks(x))

In [ ]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
#Adding adjectives
spam_df['adjectives'] = spam_df['clean_text'].apply(lambda x: ' '.join([word for word, pos in nltk.pos_tag(nltk.word_tokenize(x)) if pos.startswith('JJ')]))

In [ ]:
spam_df.head()

,label,message,clean_text,Number_of_words,Number_of_characters,Number_of_digits,Number_of_exclamation_marks,adjectives
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,20,102,0,0,jurong great buffet wat
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,6,23,0,0,lar
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,30,156,3,0,Free wkly final std
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,11,43,0,0,U early
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I do n't think he goes to usf he lives aro...,14,61,0,0,


In [ ]:
# Convert the "label" column values to numerical representation(0 for "spam",1 for "ham")

spam_df.loc[spam_df["label"] == "spam", "Category"] = 0
spam_df.loc[spam_df["label"] == "ham", "Category"] = 1

spam_df.head()

,label,message,clean_text,Number_of_words,Number_of_characters,Number_of_digits,Number_of_exclamation_marks,adjectives,Category
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,20,102,0,0,jurong great buffet wat,1.0
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,6,23,0,0,lar,1.0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,30,156,3,0,Free wkly final std,0.0
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,11,43,0,0,U early,1.0
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I do n't think he goes to usf he lives aro...,14,61,0,0,,1.0


In [ ]:
#Dropping adjectives column

spam_df.drop('adjectives', axis=1, inplace=True)

In [ ]:
spam_df.head()

,label,message,clean_text,Number_of_words,Number_of_characters,Number_of_digits,Number_of_exclamation_marks,Category
0,ham,"Go until jurong point, crazy.. Available only ...",Go until jurong point crazy Available only in ...,20,102,0,0,1.0
1,ham,Ok lar... Joking wif u oni...,Ok lar Joking wif u oni,6,23,0,0,1.0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,30,156,3,0,0.0
3,ham,U dun say so early hor... U c already then say...,U dun say so early hor U c already then say,11,43,0,0,1.0
4,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I do n't think he goes to usf he lives aro...,14,61,0,0,1.0


In [ ]:
x=spam_df.iloc[:,3:7]

In [ ]:
x.head()

,Number_of_words,Number_of_characters,Number_of_digits,Number_of_exclamation_marks
0,20,102,0,0
1,6,23,0,0
2,30,156,3,0
3,11,43,0,0
4,14,61,0,0


In [ ]:
y=spam_df.iloc[:,7:]

In [ ]:
y.head()

,Category
0,1.0
1,1.0
2,0.0
3,1.0
4,1.0


# <font color = 'indian red'> **Over Sampling using SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
y.value_counts()

Category
1.0         4825
0.0          747
dtype: int64

In [ ]:
smote = SMOTE(random_state=3)
x_balanced, y_balanced = smote.fit_resample(x,y)

In [ ]:
y_balanced.value_counts()

Category
0.0         4825
1.0         4825
dtype: int64

# <font color = 'indian red'> **Modelling the data**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x,test_x,train_y,test_y = train_test_split(x_balanced,y_balanced,test_size=0.2,random_state=3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(train_x,train_y)

RandomForestClassifier()

In [ ]:
pred_y = model.predict(test_x)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_y,pred_y))

              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94       962
         1.0       0.95      0.93      0.94       968

    accuracy                           0.94      1930
   macro avg       0.94      0.94      0.94      1930
weighted avg       0.94      0.94      0.94      1930



In [ ]:
# The Random Forest Model has a high F-1 score. Since F-1 score is our metric, we can deploy this model for spam detection